# Notebook 2: Text Classification Quality for Support Tickets

**Objective**: Handle text-specific quality issues in IT support ticket routing:
- Detect toxic/PII/informal content
- Identify ambiguous or verbose tickets
- Route tickets correctly despite quality issues

---

## Flow Diagram

```mermaid
flowchart TD
    subgraph Input["📥 Support Tickets"]
        A[Ticket Text]
        B[Category Labels]
        C[Priority]
    end

    subgraph TextQuality["📝 Text Quality Analysis"]
        D[Toxic Detection]
        E[PII Detection]
        F[Language Quality]
        G[Informality Score]
    end

    subgraph Cleanlab["🧹 Cleanlab Analysis"]
        H[get_label_quality_scores]
        I[find_label_issues]
        J[get_confidence_weighted_entropy]
    end

    subgraph YRSN["🎯 YRSN Mapping"]
        K{Text Issue Type?}
        L[Toxic/PII → N High]
        M[Informal → S Moderate]
        N[Verbose → S High]
        O[Clean → R High]
    end

    subgraph Routing["🚦 Temperature Routing"]
        P[Compute τ = 1/α]
        Q{τ Level?}
        R[🟢 GREEN: Auto-route]
        S[🟡 YELLOW: Summarize first]
        T[🔴 RED: Human review]
    end

    A --> D
    A --> E
    A --> F
    A --> G
    B --> H
    D --> K
    E --> K
    F --> K
    G --> K
    H --> K
    I --> K
    J --> K
    K --> L
    K --> M
    K --> N
    K --> O
    L --> P
    M --> P
    N --> P
    O --> P
    P --> Q
    Q -->|Low τ| R
    Q -->|Mid τ| S
    Q -->|High τ| T

    style TextQuality fill:#fff3e0
    style Cleanlab fill:#e1f5fe
    style YRSN fill:#e8f5e9
    style Routing fill:#fce4ec
```

---

**Collapse Type Focus**: DISTRACTION (verbose/unfocused tickets)

**Difficulty**: ⭐ Easy

## 1. Setup

In [ ]:
# Install dependencies
!pip install cleanlab sentence-transformers scikit-learn pandas boto3 --quiet

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

from cleanlab.filter import find_label_issues
from cleanlab.rank import get_label_quality_scores

# Import YRSN adapter
import sys
sys.path.append('../src')
from yrsn_iars.adapters.cleanlab_adapter import CleanlabAdapter, YRSNResult

print("Dependencies loaded successfully")

## 2. Load Support Ticket Data

In [ ]:
# Generate synthetic support ticket data
np.random.seed(42)
n_samples = 1000

categories = ['hardware', 'software', 'network', 'access', 'other']

# Templates for different quality levels
clean_templates = [
    "My laptop screen is not displaying properly. Error code: {code}. Please advise.",
    "Request access to {system}. Manager approval attached. User ID: {user}.",
    "Network connectivity issue in building {building}. Affecting {n} users.",
]

informal_templates = [
    "hey my laptop broke lol can u fix it asap???",
    "URGENT!!!! NEED ACCESS NOW!!!! been waiting forever!!!",
    "wifi is super slow today ugh so annoying",
]

verbose_templates = [
    "So yesterday I was working on my laptop and everything was fine but then today when I came in and turned it on the screen started flickering and I tried restarting it multiple times but it keeps happening and I really need to finish my project because the deadline is tomorrow and I don't know what to do please help me as soon as possible thank you so much.",
]

# Generate tickets
tickets = []
for i in range(n_samples):
    quality_type = np.random.choice(['clean', 'informal', 'verbose'], p=[0.6, 0.25, 0.15])
    
    if quality_type == 'clean':
        text = np.random.choice(clean_templates).format(
            code=np.random.randint(1000, 9999),
            system=np.random.choice(['SAP', 'Salesforce', 'SharePoint']),
            user=f'user_{np.random.randint(100, 999)}',
            building=np.random.choice(['A', 'B', 'C']),
            n=np.random.randint(5, 50)
        )
    elif quality_type == 'informal':
        text = np.random.choice(informal_templates)
    else:
        text = np.random.choice(verbose_templates)
    
    tickets.append({
        'ticket_id': f'TKT-{i:05d}',
        'text': text,
        'category': np.random.choice(categories),
        'priority': np.random.choice(['low', 'medium', 'high']),
        'quality_type': quality_type  # Ground truth for evaluation
    })

df = pd.DataFrame(tickets)

# Inject some label noise
noise_indices = np.random.choice(n_samples, size=int(n_samples * 0.08), replace=False)
for idx in noise_indices:
    df.loc[idx, 'category'] = np.random.choice(categories)

print(f"Generated {len(df)} support tickets")
print(f"\nCategory distribution:")
print(df['category'].value_counts())
print(f"\nQuality type distribution:")
print(df['quality_type'].value_counts())
df.head()

## 3. Text Quality Analysis

Detect text-specific quality issues that affect YRSN components.

In [ ]:
def compute_text_quality_scores(text: str) -> dict:
    """
    Compute text quality signals for YRSN mapping.
    
    In production, use AWS Comprehend or dedicated models.
    This is a simple heuristic version.
    """
    text_lower = text.lower()
    
    # Informality indicators
    informal_patterns = ['lol', 'asap', '!!!', 'plz', 'u ', 'pls', 'omg', 'brb']
    informality = sum(1 for p in informal_patterns if p in text_lower) / len(informal_patterns)
    
    # Verbosity (excessive length)
    word_count = len(text.split())
    verbosity = min(1.0, max(0, (word_count - 50) / 100))  # Scale 50-150 words
    
    # ALL CAPS detection (urgency/frustration)
    caps_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
    
    # PII patterns (very basic)
    pii_patterns = ['ssn', 'social security', 'password', 'credit card']
    has_pii = any(p in text_lower for p in pii_patterns)
    
    # Toxic patterns (very basic, use proper model in production)
    toxic_patterns = ['stupid', 'idiot', 'hate', 'worst']
    toxicity = any(p in text_lower for p in toxic_patterns)
    
    return {
        'informality': informality,
        'verbosity': verbosity,
        'caps_ratio': caps_ratio,
        'has_pii': has_pii,
        'toxicity': toxicity,
        'word_count': word_count
    }

# Apply to all tickets
quality_scores = df['text'].apply(compute_text_quality_scores).apply(pd.Series)
df = pd.concat([df, quality_scores], axis=1)

print("Text Quality Statistics:")
print(quality_scores.describe())

## 4. Generate Embeddings and Cleanlab Analysis

In [ ]:
# Load sentence transformer
encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
print("Generating embeddings...")
embeddings = encoder.encode(df['text'].tolist(), show_progress_bar=True)

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(df['category'])

print(f"Classes: {le.classes_}")

In [ ]:
# Train classifier and get cross-validated predictions
clf = LogisticRegression(max_iter=1000, n_jobs=-1)

print("Getting cross-validated predictions...")
pred_probs = cross_val_predict(
    clf,
    embeddings,
    labels,
    cv=5,
    method='predict_proba',
    n_jobs=-1
)

# Get Cleanlab scores
label_quality = get_label_quality_scores(labels, pred_probs)
df['label_quality'] = label_quality

label_issues = find_label_issues(labels, pred_probs)
df['is_label_issue'] = label_issues

print(f"Found {label_issues.sum()} label issues")

## 5. YRSN Decomposition with Text Quality

In [ ]:
# Initialize YRSN adapter
adapter = CleanlabAdapter()

# Compute YRSN for each ticket
def compute_ticket_yrsn(row):
    """Combine label quality with text quality for YRSN."""
    
    # Get base YRSN from Cleanlab signals
    pred_prob = pred_probs[row.name]
    entropy = -np.sum(pred_prob * np.log(pred_prob + 1e-10))
    max_entropy = np.log(len(pred_prob))
    normalized_entropy = entropy / max_entropy
    
    sorted_probs = np.sort(pred_prob)
    margin = sorted_probs[-1] - sorted_probs[-2]
    normalized_margin = (margin + 1) / 2
    
    base_yrsn = adapter.example_to_yrsn(
        label_quality=row['label_quality'],
        normalized_margin=normalized_margin,
        normalized_entropy=normalized_entropy
    )
    
    # Apply text quality adjustments
    text_yrsn = adapter.text_quality_to_yrsn(
        toxic_score=1.0 if row['toxicity'] else 0.0,
        pii_score=1.0 if row['has_pii'] else 0.0,
        informal_score=row['informality']
    )
    
    # Combine (weighted average)
    combined_R = 0.6 * base_yrsn.R + 0.4 * text_yrsn.R
    combined_S = 0.6 * base_yrsn.S + 0.4 * text_yrsn.S + 0.2 * row['verbosity']  # Verbosity → S
    combined_N = 0.6 * base_yrsn.N + 0.4 * text_yrsn.N
    
    # Normalize
    total = combined_R + combined_S + combined_N
    return YRSNResult(R=combined_R/total, S=combined_S/total, N=combined_N/total)

# Apply YRSN computation
yrsn_results = df.apply(compute_ticket_yrsn, axis=1)
df['R'] = [y.R for y in yrsn_results]
df['S'] = [y.S for y in yrsn_results]
df['N'] = [y.N for y in yrsn_results]
df['collapse_type'] = [y.collapse_type.value for y in yrsn_results]

print("YRSN Statistics:")
print(df[['R', 'S', 'N']].describe())

## 6. Temperature-Based Routing

In [ ]:
from yrsn_iars.adapters.temperature import compute_temperature, TemperatureMode

# Compute temperature for each ticket
df['temperature'] = df['R'].apply(lambda r: compute_temperature(r))

# Determine routing stream
def determine_stream(row):
    tau = row['temperature']
    R = row['R']
    N = row['N']
    
    # Collapse check
    if row['collapse_type'] in ['poisoning', 'confusion']:
        return 'red'
    
    # Temperature-adjusted routing
    if tau < 1.2 and R > 0.6 and N < 0.2:
        return 'green'
    elif tau < 2.0 and R > 0.4:
        return 'yellow'
    else:
        return 'red'

df['stream'] = df.apply(determine_stream, axis=1)

print("Routing Distribution:")
print(df['stream'].value_counts())
print(f"\nAutomation rate: {100 * (df['stream'] == 'green').mean():.1f}%")

## 7. Analyze Routing by Text Quality

In [ ]:
# Cross-tabulate quality type with routing stream
routing_by_quality = pd.crosstab(
    df['quality_type'],
    df['stream'],
    normalize='index'
).round(3) * 100

print("Routing Distribution by Text Quality Type (%):")
print(routing_by_quality)

# Average YRSN by quality type
print("\nAverage YRSN by Quality Type:")
print(df.groupby('quality_type')[['R', 'S', 'N', 'temperature']].mean().round(3))

## 8. Examine DISTRACTION Cases (High S)

In [ ]:
# Find tickets with high S (superfluous/verbose)
distraction_cases = df[df['collapse_type'] == 'distraction'].nlargest(10, 'S')

print("Top 10 DISTRACTION Cases (High S):")
print("="*80)
for _, row in distraction_cases.iterrows():
    print(f"\n[{row['ticket_id']}] R={row['R']:.2f}, S={row['S']:.2f}, N={row['N']:.2f}")
    print(f"Stream: {row['stream'].upper()}, τ={row['temperature']:.2f}")
    print(f"Text: {row['text'][:150]}..." if len(row['text']) > 150 else f"Text: {row['text']}")

## 9. Export Results

In [ ]:
# Save for next notebook
output_cols = ['ticket_id', 'text', 'category', 'priority', 
               'R', 'S', 'N', 'collapse_type', 'temperature', 'stream',
               'label_quality', 'informality', 'verbosity']

df[output_cols].to_csv('ticket_yrsn_results.csv', index=False)
print(f"Saved {len(df)} tickets with YRSN analysis")

## Summary

In this notebook we:
1. Loaded support ticket data with various text quality issues
2. Computed text-specific quality signals (informality, verbosity, toxicity)
3. Combined Cleanlab label quality with text quality for YRSN decomposition
4. Applied temperature-based routing (τ = 1/α)
5. Analyzed routing patterns by text quality type
6. Identified DISTRACTION collapse cases (high S)

**Key Insight**: Verbose/informal text increases S (superfluous), which raises temperature and routes to yellow/red for human summarization before processing.

**Next**: Notebook 3 - Multi-Annotator Consensus for Committee Decisions